In [3]:
from selenium import webdriver

driver = webdriver.Chrome("chromedriver") # 윈도우라면 chromedriver.exe

In [4]:
driver.get("https://google.com")

In [5]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.46M/6.46M [00:02<00:00, 2.27MB/s]


In [6]:
import time
from selenium.webdriver.common.by import By

driver.get("https://google.com")
element = driver.find_element(
    By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
)

element.send_keys(
    "Python\n",
)
time.sleep(1)
assert driver.current_url.startswith("https://www.google.com/search?q=Python")

In [7]:
keyword = "떡볶이"

base_url = f"https://www.youtube.com/results?search_query={keyword}&sp=EgIQAQ%253D%253D"
driver.get(base_url)
time.sleep(10)

In [8]:
base_xpath = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer/div[1]"
video_element = driver.find_element('xpath', base_xpath)

In [9]:
video_xpath = "./div/div[1]/div/h3/a"
video = video_element.find_element('xpath', video_xpath)
video_title = video.text

In [10]:
channel_xpath = "./div/div[2]/ytd-channel-name/div/div/yt-formatted-string/a"
channel_name = video_element.find_element('xpath', channel_xpath).text

In [11]:
# get url, but skip shorts
url = video.get_attribute("href")

print(url, channel_name, video_title)

https://www.youtube.com/watch?v=t4Es8mwdYlE 백종원 PAIK JONG WON 분식집st 떡 볶 이🎉


In [13]:
import time

# Scroll to the bottom
stop_xpath = '//*[@id="message"]'
scroll = 10000
idx = 0
while True:
    if driver.find_element('xpath', stop_xpath).text in ["No more results", "결과가 더 이상 없습니다."]:
        break
    driver.execute_script(f"window.scrollTo(0, {scroll});")
    time.sleep(2)

    scroll += 10000

KeyboardInterrupt: 

In [14]:
# Extract data
data = {}
base_xpath = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer/div[1]"
video_elements = driver.find_elements('xpath', base_xpath)

for video_element in video_elements:
    video_xpath = "./div/div[1]/div/h3/a"
    video = video_element.find_element('xpath', video_xpath)
    video_title = video.text

    # get channel name
    channel_xpath = (
        "./div/div[2]/ytd-channel-name/div/div/yt-formatted-string/a"
    )
    channel_name = video_element.find_element('xpath', channel_xpath).text

    url = video.get_attribute("href")

    data[video_title] = {
        "channel_name": channel_name,
        "url": url,
    }

data

{'': {'channel_name': '',
  'url': 'https://www.youtube.com/watch?v=MHG2D7AcazY'}}

In [15]:
import time

driver.get('https://datavalue.dunamu.com/feargreedindex')
time.sleep(3)

In [ ]:
import time

from tqdm import tqdm
from selenium_helper import SeleniumBase
import pandas as pd

# initialized the driver and go to the url
sb = SeleniumBase()
sb.setup_driver()
base_url = 'https://datavalue.dunamu.com/feargreedindex'
sb.driver.get(base_url)
time.sleep(3)

# find the last page number and iterate over
last_page_xpath = (
    '/html/body/div[2]/div/div[1]/div/div/div[4]/div/div[3]/span/a[6]'
)
last_page_num = int(sb.driver.find_element('xpath', last_page_xpath).text)

result = pd.DataFrame()
next_button_xpath = '//*[@id="table-series_next"]'
for _ in tqdm(range(1, last_page_num)):
    # extract HTML table from the page
    table_body_xpath = '//*[@id="table-series"]'
    table_body_element = sb.driver.find_element("xpath", table_body_xpath)

    df = pd.read_html(table_body_element.get_attribute('outerHTML'))[0]
    result = pd.concat([result, df])

    # go to the next page
    sb.click(next_button_xpath)